In [1]:
import os
import sys
sys.path.append("../")
import tensorflow as tf
from glob import glob
from typing import List, Tuple
#from .data_processing.data_loader import dataset_loader
#from rosbag2numpy.data_processing.data_loader_costmap  import dataset_loader
from rosbag2numpy.data_processing.data_loader_dual_costmap  import dataset_loader

from matplotlib import pyplot as plt
import numpy as np
from numpy import ndarray
from typing import Dict, List, Union
from rosbag2numpy.config import params
from rosbag2numpy.config import generalization_model_params as g_params
import wandb
from wandb.keras import WandbCallback
from rosbag2numpy.losses import euclidean_distance_loss,enc_costmap_loss
from rosbag2numpy.models import conv1x1_endpoint_in_model,conv1x1_endpoint_in_model_costmapout, LSTMconv1x1_endpoint_in_model, LSTMconv1x1_with_encoder
import time
import os
from rosbag2numpy.train import get_np_test_ds
from tensorflow.keras import backend as K
from rosbag2numpy.train import get_np_test_ds, cd_wand_custom

print(tf.__version__)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


2.3.0
2.3.0


In [2]:
ds_loader = dataset_loader(
    tfrec_dir=params.get("dataset_dir"),
    batch_size=params.get("H_BATCH_SIZE"),
    shuffle_buffer=params.get("H_SHUFFLE_BUFFER"),
    normalize_coords=params.get("normalize_coords"),
    normalize_factor=params.get("normalize_factor")
)
ds_train, ds_valid, ds_test = ds_loader.build_scenario_dataset(consider_scenes=10,no_train_scene=8,no_valid_scene=1,no_test_scene=1)

start = time.time()
#np_ds_test = get_np_test_ds(ds_test=ds_test)
print(f"Test dataset load time: {time.time() - start}")

Scenarios:['/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario1', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario2', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario3', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario4', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario5', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_11', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_12', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_13', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_14', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario_ph_15']
Building Dataset.......

Number of records, Train files:610, validation files:109, Test Files:

In [3]:
"""
for X,Y in ds_train.take(1):
    #print(X[-1])
    # costmap, optimized path gt
    print(Y.shape)
    # costmap, grid org res, left bnd, right bnd, car odo, init path, filedetails, gt opt path cost
    print(X[0].shape,X[1].shape,X[2].shape,X[3].shape,X[4].shape,X[5].shape,X[6].shape)
"""

#X[0][0][:,:]

'\nfor X,Y in ds_train.take(1):\n    #print(X[-1])\n    # costmap, optimized path gt\n    print(Y.shape)\n    # costmap, grid org res, left bnd, right bnd, car odo, init path, filedetails, gt opt path cost\n    print(X[0].shape,X[1].shape,X[2].shape,X[3].shape,X[4].shape,X[5].shape,X[6].shape)\n'

In [4]:
#model = conv1x1_endpoint_in_model_costmapout.nn(full_skip= True,params=params)
model = LSTMconv1x1_with_encoder.nn(full_skip = True, params = params)
# Learning rate scheduler

#lrs = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=params.get("lr"), decay_steps=params.get("lr_decay_steps"), decay_rate=params.get("lr_decay_rate"))
#learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=.02, decay_steps=20, decay_rate=.1)
opt = LSTMconv1x1_with_encoder._get_optimizer(params.get("optimizer"), lr=params.get("lr"))

model.compile(
    optimizer=opt,
    loss=params.get("losses"),
    loss_weights=params.get("loss_weights"),
    metrics=params.get("metric")
)

cb_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2, patience=3, min_lr=0.0001
)
#model.summary()

Model: "functional_1"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   [(None, 1536, 1536, 1)]    0                                                     
________________________________________________________________________________________________________________________
Left_boundary (InputLayer)             [(None, 25, 2)]            0                                                     
________________________________________________________________________________________________________________________
Right_boundary (InputLayer)            [(None, 25, 2)]            0                                                     
________________________________________________________________________________________________________________________
Initial_pa

In [5]:

history = model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_valid,
    callbacks=[cb_reduce_lr],
)

(None, 50, 2)
(None, 25, 2)
(None, 25, 2)


ValueError: in user code:

    /netpool/work/gpu-3/users/malyalasa/anaconda3/envs/env_mt/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /netpool/work/gpu-3/users/malyalasa/anaconda3/envs/env_mt/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /netpool/work/gpu-3/users/malyalasa/anaconda3/envs/env_mt/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /netpool/work/gpu-3/users/malyalasa/anaconda3/envs/env_mt/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /netpool/work/gpu-3/users/malyalasa/anaconda3/envs/env_mt/lib/python3.8/site-packages/ten

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Loss over epochs")
plt.plot(loss_index,loss_list)
plt.plot(loss_index,[max(loss_list) if i%287==0 else 0 for i in range(len(loss_list))])
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
from tqdm import tqdm

# [ip_gridmap,ip_grid_org_res,ip_left_bnd, ip_right_bnd, ip_car_odo, ip_init_path,ip_file_name,ip_gt_cost]

#@tf.function(experimental_relax_shapes=True)
def train_step(data,step,epoch):
    #(costmap, grid_org_res, left_bnd, right_bnd, car_odo, init_path, filedetails),(gt_opt_path) = data
    X,Y = data

    #print(X[0].shape,X[1].shape,X[2].shape,X[3].shape,X[4].shape,X[5].shape,X[6].shape)
    with tf.GradientTape() as tape:

        enc_grid,predictions = model(X,training=True)

        #cm_loss_pred = path_costmap_loss(X[0],predictions)
        #cm_loss_gt = path_costmap_loss(X[0],Y)

        displacement_loss = euclidean_distance_loss(Y,predictions)
        
        if epoch > 14:
            path_cost_loss = K.sum(K.flatten(tf.abs(enc_grid * tf.abs(Y-predictions)))) #tf.math.reduce_mean(tf.abs(cm_loss_pred-cm_loss_gt))

            total_loss = displacement_loss#path_cost_loss + displacement_loss
        else:
            total_loss = displacement_loss
        #print(total_loss)

        grads = tape.gradient(total_loss, model.trainable_weights)

        opt.apply_gradients(zip(grads,model.trainable_weights))
    return total_loss

#@tf.function(experimental_relax_shapes=True)
def train():
    epochs = 40
    step_count = 0
    loss_index = []
    loss_list = []
    for epoch in range(epochs):
        print(f"\n Start of epoch: {epoch+1}/{epochs}")

        #iterate through the batches of dataset.
        for step, (X_batch,Y_batch) in enumerate(tqdm(ds_train)):
            loss = train_step((X_batch,Y_batch),step,epoch)
            loss_list.append(loss)
            step_count+=step
            loss_index.append(step_count)
        
        print(f"Loss at epoch {epoch+1} is {loss_list[-1]}")

        if epoch==30:
            model.save('trainmodel.h5')

    return loss_index,loss_list

loss_index, loss_list= train()

In [4]:
from concurrent.futures import ProcessPoolExecutor, as_completed


def conc_path_costmap_loss(y_true, y_pred):
    batch_loss = []
    #print(f'shapes: {type(y_true.shape[0]),y_pred.shape}')
    assert y_true.shape[0] == y_pred.shape[0]

    # range of batch size, last batch may not have samples mentioned in batch size
    #for i in range(y_true.shape[0]):
    #    batch_loss.append(ind_loss(y_true[i][:,:], y_pred[i][:,:])) 

    with ProcessPoolExecutor() as executor:
        for i in range(y_true.shape[0]):
            param_args = {'y_true':y_true[i][:,:],'y_pred': y_pred[i][:,:]}
            results = {executor.submit(ind_loss,**param_args)}
        for future in as_completed(results):
            batch_loss.append(future.result())

    executor.shutdown(wait=True)
    #print(batch_loss)
    #[batch_loss.append(result) for result in results]
    return tf.convert_to_tensor(batch_loss)


def path_costmap_loss(y_true, y_pred):
    batch_loss = []
    #print(f'shapes: {type(y_true.shape[0]),y_pred.shape}')
    assert y_true.shape[0] == y_pred.shape[0]

    # range of batch size, last batch may not have samples mentioned in batch size
    for i in range(y_true.shape[0]):
        #batch_loss.append(ind_loss(y_true[i][:,:], y_pred[i][:,:])) 
        batch_loss.append(ind_loss(y_true[i][:,:], y_pred[i][:,:])) 

    #print(batch_loss)
    return tf.convert_to_tensor(batch_loss)

def ind_loss(y_true, y_pred):
    #assuming y true is cost map 1536,1536
    # y_pred is a list of predicted points 25,2

    costmap = y_true
    valid_indices = tf.where(costmap > 0.35) # get indices
    valid_costs = tf.gather_nd(costmap, valid_indices) # get respective values
    valid_costs = tf.cast(valid_costs, dtype=tf.float32) #cast them to float32

    
    allcosts = tf.constant(0.0)
    valid_indices = tf.cast(valid_indices, dtype=tf.float32)
    #print(f"valid_costs:{valid_costs.shape}, valid indices shape : {valid_indices.shape}")

    for i in range(25):

        pred_dist  = K.sqrt(K.sum(K.square(valid_indices - y_pred[i]),axis=1))
        inv_pred_dist = tf.math.reciprocal_no_nan(tf.math.pow(pred_dist,0.1))

        # cost for one point in the path
        cost_for_point = tf.reduce_sum(tf.multiply(inv_pred_dist,valid_costs))
        pred_allcosts = tf.add(allcosts,cost_for_point) if i == 0 else tf.add(pred_allcosts,cost_for_point)
    #print(pred_allcosts.shape)
    
    return K.mean(pred_allcosts)
 

In [ ]:
opt = conv1x1_endpoint_in_model._get_optimizer(params.get("optimizer"), lr=params.get("lr"))
lstm_model.compile(
    optimizer=opt, 
    loss=params.get("losses"),
    loss_weights=params.get("loss_weights"), metrics=params.get("metric")
)
    
# Learning rate scheduler
cb_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2, patience=3, min_lr=0.0001
)

#tensorboardcallback = tf.keras.callbacks.TensorBoard(log_dir=params.get("log_dir"),profile_batch=2)

wandb.init(project="ppmodel_base", config=params)
np_ds_test = get_np_test_ds(ds_test=ds_test)
# Model training
history = lstm_model.fit(
    ds_train,
    epochs=params.get("epochs"),
    validation_data=ds_valid,
    callbacks=[
            cb_reduce_lr,
            #WandbCallback(),
            cd_wand_custom(ds_test=ds_test, 
            np_test_dataset=np_ds_test,
            test_index=40,
            normalized_coords=params.get("normalize_coords"),
            normalize_factor = params.get("normalize_factor")
            
            )
        ],
)